<a href="https://colab.research.google.com/github/aditya26091999/music-genre-classification/blob/main/music-genre-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Google Collab is meant to house the code for DMW-Mini-Project. The project deals with classifying the songs into respective genres.The datasets that can be used include Spotify, Gaana, Wynk, Shazam, Saavn, etc.Yet another audio dataset that can be used is GTZAN dataset.

The dataset will be trained on some of the state-of-the-art classification algorithms like, SVM, Logistic Regression, CNN, Decision tree etc.

The best algorithm choice will be made, after proper examination of results, accuracy and other deciding factors

In [ ]:
print('Welcome to the MUSIC-GENRE-CLASSIFICATION Project!')

In [ ]:
#Downloading some Google-Collab exclusive dependencies
!pip install flask-ngrok

# Importing the necessary libraries for the project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh as bk
import plotly as plt
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify,request
from IPython.display import display

# **Music-Genre-Classification** Dataset Insights

In [ ]:
#loading the dataset from Github folder
datasetUrl = 'https://raw.githubusercontent.com/aditya26091999/music-genre-classification/main/Probable%20Datasets/26-Genres.csv'
dataset = pd.read_csv(datasetUrl)
dataset.set_index("genre",inplace=True)

Inspecting the dataset structure and properties

In [ ]:
#checking the dataset loaded from Github
display(dataset.head())


In [ ]:
#displaying the unique genres[Classes for classification] present
display(dataset.index.unique())
print()

#choosing only required genres(classes)
genres_id = ['Country', 'Dance', 'Electronic', 'Folk', 'Hip-Hop', 'Rap', 'Classical', 'Pop', 'Jazz', 'Rock']
print("Classes for the model : ",genres_id)

In [ ]:
#cleaning the dataset for unwanted genre classes and their associated rows
display(dataset.columns)
display(dataset.index.unique())

columns_to_drop = ['artist_name', 'track_name', 'track_id', 'duration_ms','popularity', 'mode','time_signature']

rows_to_drop = ['Movie', 'R&B', 'A Capella', 'Alternative', 'Anime','Blues', 'Opera',"Children's Music", 'Children’s Music', 'Indie', 'Reggae', 'Reggaeton', 'Ska', 'Comedy', 'Soul','Soundtrack', 'World']

# Dropping the unwanted rows and columns
dataset.drop(columns_to_drop,axis=1,inplace=True)
dataset.drop(rows_to_drop,inplace=True)

In [109]:
#Final cleaned dataset
display(dataset)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
genre,,,,,,,,,,
Country,0.00821,0.551,0.704,0.000002,G#,0.2450,-5.426,0.0444,97.075,0.541
Country,0.02720,0.375,0.859,0.000000,G,0.2670,-3.243,0.0569,74.059,0.597
Country,0.69000,0.699,0.408,0.000219,B,0.1450,-11.460,0.0960,171.922,0.842
Country,0.15200,0.555,0.726,0.000178,D,0.1480,-8.939,0.0368,107.719,0.505
Country,0.57100,0.640,0.478,0.000000,B,0.0902,-6.960,0.0330,136.717,0.315
...,...,...,...,...,...,...,...,...,...,...
Rock,0.67000,0.718,0.593,0.000000,C,0.1720,-9.976,0.0377,124.251,0.869
Rock,0.03400,0.521,0.933,0.000021,B,0.3300,-4.874,0.1790,169.842,0.607
Rock,0.67100,0.618,0.769,0.241000,B,0.2760,-4.642,0.0309,111.978,0.607
